In [2]:
import pandas as pd
import requests
from datetime import datetime
import pytz

In [ ]:
city = 'Berlin'
API_key = '4fe53ee5e34a7d900ed58bd74bbbb0b7'

# check out the docs for more info on making an api call https://openweathermap.org/forecast5
url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")

response = requests.get(url)
json = response.json()

json

Now we've discovered what information we have to work with. Let's decide what we want to keep and what we wish to lose.

I feel that from json['list'] it would be good to keep

1.'weather.main', 'weather.description', 'dt_txt', 'main.temp', 'main.feels_like' 'clouds.all', 'rain.3h', 'snow.3h' 'wind.speed', 'wind.deg', 'main.humidity', 'main.pressure'.
And from json['city'] it would be good to keep

2.'name' and 'country.

Just to make sure that we got the right place. And as an added extra we'll also include the time that API call was made, so we know how up to date our forecast is.


**Optional**: let's get a timestamp of when we get the data. Datetime uses the uses the current time of the system, which on local computers is normally correct. But as we're in the cloud, computers are not always in our country, and we therefore add on the timezone module to ensure that our timestamp is local to us and not the computer.
****

In [4]:
tz = pytz.timezone('Europe/Berlin')
now = datetime.now().astimezone(tz)

now

datetime.datetime(2022, 7, 16, 23, 46, 27, 338081, tzinfo=<DstTzInfo 'Europe/Berlin' CEST+2:00:00 DST>)

Next let's loop through the json['list'] information get the weather information



In [5]:
# we'll store the information in this dicitonary:
weather_dict = {'city': [],
                'country': [],
                'forecast_time': [],
                'outlook': [],
                'detailed_outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'clouds': [],
                'rain': [],
                'snow': [],
                'wind_speed': [],
                'wind_deg': [],
                'humidity': [],
                'pressure': []}
                #'information_retrieved_at': []}

# let's begin the loop
for i in json['list']:
  weather_dict['city'].append(json['city']['name'])
  weather_dict['country'].append(json['city']['country'])
  weather_dict['forecast_time'].append(i['dt_txt'])
  weather_dict['outlook'].append(i['weather'][0]['main'])
  weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
  weather_dict['temperature'].append(i['main']['temp'])
  weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
  weather_dict['clouds'].append(i['clouds']['all'])
  # sometimes the data is missing for rain and snow. As it is not always raining or snowing
  # we cannot make a DataFrame unless the lists are all the same length, therefore missing values are bad
  # here we say try to append a value if there is one. If not, append a 0
  try:
      weather_dict['rain'].append(i['rain']['3h'])
  except:
      weather_dict['rain'].append('0')
  try:
      weather_dict['snow'].append(i['snow']['3h'])
  except:
      weather_dict['snow'].append('0')
  weather_dict['wind_speed'].append(i['wind']['speed'])
  weather_dict['wind_deg'].append(i['wind']['deg'])
  weather_dict['humidity'].append(i['main']['humidity'])
  weather_dict['pressure'].append(i['main']['pressure'])
  #weather_dict['information_retrieved_at'].append(now.strftime("%d/%m/%Y %H:%M:%S"))
  

Now we convert our dictionary to a DataFrame


In [6]:
weather_from_dict_df = pd.DataFrame(weather_dict)

As a final step, to keep everything tidy: let's bring everything we did together in a function. And allow the function to take a list of cities as an input


In [7]:
def get_weather_loop(cities):

  API_key = '4fe53ee5e34a7d900ed58bd74bbbb0b7'

  tz = pytz.timezone('Europe/Berlin')
  now = datetime.now().astimezone(tz)

  weather_dict = {'city': [],
                'country': [],
                'forecast_time': [],
                'outlook': [],
                'detailed_outlook': [],
                'temperature': [],
                'temperature_feels_like': [],
                'clouds': [],
                'rain': [],
                'snow': [],
                'wind_speed': [],
                'wind_deg': [],
                'humidity': [],
                'pressure': [],
                'information_retrieved_at': []}

  for city in cities:
    url = (f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_key}&units=metric")
    response = requests.get(url)
    json = response.json()

    for i in json['list']:
      weather_dict['city'].append(json['city']['name'])
      weather_dict['country'].append(json['city']['country'])
      weather_dict['forecast_time'].append(i['dt_txt'])
      weather_dict['outlook'].append(i['weather'][0]['main'])
      weather_dict['detailed_outlook'].append(i['weather'][0]['description'])
      weather_dict['temperature'].append(i['main']['temp'])
      weather_dict['temperature_feels_like'].append(i['main']['feels_like'])
      weather_dict['clouds'].append(i['clouds']['all'])
      try:
          weather_dict['rain'].append(i['rain']['3h'])
      except:
          weather_dict['rain'].append('0')
      try:
          weather_dict['snow'].append(i['snow']['3h'])
      except:
          weather_dict['snow'].append('0')
      weather_dict['wind_speed'].append(i['wind']['speed'])
      weather_dict['wind_deg'].append(i['wind']['deg'])
      weather_dict['humidity'].append(i['main']['humidity'])
      weather_dict['pressure'].append(i['main']['pressure'])
      weather_dict['information_retrieved_at'].append(now.strftime("%d/%m/%Y %H:%M:%S"))

  return pd.DataFrame(weather_dict)

In [9]:
get_weather_loop(['Berlin', 'London','Stuttgart'])

,city,country,forecast_time,outlook,detailed_outlook,temperature,temperature_feels_like,clouds,rain,snow,wind_speed,wind_deg,humidity,pressure,information_retrieved_at
0,Berlin,DE,2022-07-17 00:00:00,Clear,clear sky,14.89,13.98,4,0,0,3.40,303,59,1022,16/07/2022 23:51:24
1,Berlin,DE,2022-07-17 03:00:00,Clear,clear sky,12.37,11.49,5,0,0,3.10,285,70,1024,16/07/2022 23:51:24
2,Berlin,DE,2022-07-17 06:00:00,Clouds,few clouds,13.47,12.57,14,0,0,3.62,286,65,1026,16/07/2022 23:51:24
3,Berlin,DE,2022-07-17 09:00:00,Clouds,broken clouds,18.07,17.13,84,0,0,3.80,289,46,1026,16/07/2022 23:51:24
4,Berlin,DE,2022-07-17 12:00:00,Clouds,overcast clouds,22.38,21.51,90,0,0,4.24,293,32,1025,16/07/2022 23:51:24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,Stuttgart,DE,2022-07-21 09:00:00,Clear,clear sky,26.37,26.37,0,0,0,0.60,202,50,1019,16/07/2022 23:51:24
116,Stuttgart,DE,2022-07-21 12:00:00,Clear,clear sky,30.24,29.52,5,0,0,1.52,239,36,1017,16/07/2022 23:51:24
117,Stuttgart,DE,2022-07-21 15:00:00,Clouds,few clouds,31.18,30.12,15,0,0,3.72,276,32,1017,16/07/2022 23:51:24
118,Stuttgart,DE,2022-07-21 18:00:00,Clear,clear sky,27.97,27.92,10,0,0,1.42,295,44,1016,16/07/2022 23:51:24
